<a href="https://colab.research.google.com/github/NagaJahnaviD/WildSightAI/blob/main/WildSightAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import random
import shutil

# Path to your original dataset
original_dataset_dir = '/content/drive/My Drive/archive/animals/animals'

# Paths where you want to save train and validation splits
train_dir = '/content/drive/My Drive/animal_habit_data_split/train/'
val_dir = '/content/drive/My Drive/animal_habit_data_split/validation/'

# How much for validation (e.g., 0.2 = 20%)
val_split = 0.2

# Make sure train and val folders exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Loop over each class folder in the original dataset
for class_name in os.listdir(original_dataset_dir):
    class_path = os.path.join(original_dataset_dir, class_name)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        random.shuffle(images)

        # Split
        val_size = int(len(images) * val_split)
        val_images = images[:val_size]
        train_images = images[val_size:]

        # Create class folders in train and val directories
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        # Copy images to the train directory
        for img_name in train_images:
            src_path = os.path.join(class_path, img_name)
            dst_path = os.path.join(train_dir, class_name, img_name)
            shutil.copy(src_path, dst_path)

        # Copy images to the validation directory
        for img_name in val_images:
            src_path = os.path.join(class_path, img_name)
            dst_path = os.path.join(val_dir, class_name, img_name)
            shutil.copy(src_path, dst_path)

print("✅ Dataset split completed!")


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Update this path according to your drive
train_dir = '/content/drive/My Drive/animal_habit_data_split/train/'
val_dir = '/content/drive/My Drive/animal_habit_data_split/validation/'

# Preprocessing setup
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Split for training and validation
)

train_generator = train_datagen.flow_from_directory(
    train_dir,  # Use the new train data directory
    target_size=(224, 224),  # Size MobileNet expects
    batch_size=32,
    class_mode='categorical',
)

val_generator = train_datagen.flow_from_directory(
    val_dir,  # Use the new validation data directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
)


Found 4411 images belonging to 90 classes.
Found 1098 images belonging to 90 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Define the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model (don’t train the pretrained layers)
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,433,562 (9.28 MB)

 Trainable params: 175,578 (685.85 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from tensorflow.keras.models import load_model  # Add this import

# Model path where the trained model is saved
model_path = '/content/drive/My Drive/animal_habit_model_mobilenet.h5'

if os.path.exists(model_path):
    print("Loading existing model...")
    model = load_model(model_path)
    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(val_generator)
    print(f"✅ Validation Loss: {loss:.4f}")
    print(f"✅ Validation Accuracy: {accuracy:.4f}")
else:
    print("No existing model found, ready to train new model.")


Loading existing model...


35/35 ━━━━━━━━━━━━━━━━━━━━ 256s 7s/step - accuracy: 0.9756 - loss: 0.0897
✅ Validation Loss: 0.1100
✅ Validation Accuracy: 0.9754


In [ ]:
from tensorflow.keras.models import load_model

# If no model is saved, train a new one
if not os.path.exists(model_path):
    print("Training new model...")
    history = model.fit(
        train_generator,
        epochs=5,
        validation_data=val_generator
    )
    # Save the trained model
    model.save(model_path)
    print("Model saved!")


In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np
import io
from PIL import Image

# Upload image
uploaded = files.upload()

# Get the uploaded filename and preprocess the image
for filename in uploaded.keys():
    # Load and convert to RGB (fixes RGBA issue!)
    img = Image.open(io.BytesIO(uploaded[filename])).convert("RGB")
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict the class
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)

    # Map index to class name
    class_names = list(train_generator.class_indices.keys())
    print(f"✅ Predicted class for {filename}: {class_names[predicted_class]}")


Saving WIN_20240903_14_12_15_Pro.jpg to WIN_20240903_14_12_15_Pro.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
✅ Predicted class for WIN_20240903_14_12_15_Pro.jpg: goldfish


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_animal_info(animal_name):
    try:
        # Format search URL for Wikipedia
        search_url = f"https://en.wikipedia.org/wiki/{animal_name.capitalize()}"
        response = requests.get(search_url)

        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find first paragraph
        paragraphs = soup.select('p')
        for para in paragraphs:
            text = para.get_text().strip()
            if text:
                return f"📚 About {animal_name.capitalize()}:\n\n{text}"
        return "❗No description found."
    except Exception as e:
        return f"⚠️ Error fetching info: {e}"

# Use the predicted class from previous cell
animal_name = class_names[predicted_class]
print(get_animal_info(animal_name))


📚 About Elephant:

Elephants are the largest living land animals. Three living species are currently recognised: the African bush elephant (Loxodonta africana), the African forest elephant (L. cyclotis), and the Asian elephant (Elephas maximus). They are the only surviving members of the family Elephantidae and the order Proboscidea; extinct relatives include mammoths and mastodons. Distinctive features of elephants include a long proboscis called a trunk, tusks, large ear flaps, pillar-like legs, and tough but sensitive grey skin. The trunk is prehensile, bringing food and water to the mouth and grasping objects. Tusks, which are derived from the incisor teeth, serve both as weapons and as tools for moving objects and digging. The large ear flaps assist in maintaining a constant body temperature as well as in communication. African elephants have larger ears and concave backs, whereas Asian elephants have smaller ears and convex or level backs.
